In [ ]:
#general definitions and imports
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
%load_ext itikz

import logging
logging.Logger.disabled=True

import os
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import numpy as np
import matplotlib.pyplot as plt

import ipytest
ipytest.autoconfig()

from pathlib import Path

from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Spice.Netlist import Circuit
from PySpice.Probe.Plot import plot
from PySpice.Math import *

import schemdraw
import schemdraw.elements as elm
from schemdraw import dsp
from schemdraw import logic

## setup spice library path
directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
%%run_pytest[clean] -qq --json=build/pegel-test.json

# define the tests

def test_my_func():
    assert 0 == 0
    assert 1 == 0
    assert 2 == 2
    assert 3 == 2


Pegel is a voltage controlled amplifier (VCA). There are various designs to implement a VCA. rod elliott (ESP) has an article on VCA techniques [[1][1]]. diy synthesizer modules are usualy designed with an integrated VCA chip [[2][2]] or with a differential amplifier [[3][3]]. The integrated VCA chips are either obsolete or rather expensive. The differential amplifier can be built with transistors only. the design has some downsides. Even in the simulation the result is not symetryc and has a dc offset from the control voltage. This needs a lot of trimming to get an accurate result. but the biggest downside is, that this design can not do proper amplitude modulation (am). when the carrier signal goes below zero the base signal is completely muted.
{: class="mt-6"}  

# construction
{: class="subtitle is-lowercase"}

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 


In [ ]:
d = schemdraw.Drawing(inches_per_unit=0.6)

d.add(elm.Line( "right", l=d.unit/4, xy=[0, 0*d.unit], lftlabel='$IN_4$'))
D1 = d.add(dsp.Square(label='$\int$'))
d.add(elm.Line( "right", l=d.unit/8))
d.add(elm.Line( "up", l=0.75*d.unit))

d.add(elm.Line( "right", l=d.unit/4, xy=[0, 0.5*d.unit], lftlabel='$IN_3$'))
D2 = d.add(dsp.Square(label='$\int$'))
d.add(elm.Line( "right", l=d.unit/8))

d.add(elm.Line( "right", l=d.unit/4, xy=[0, 1*d.unit], lftlabel='$IN_2$'))
D3 = d.add(dsp.Square(label='$\int$'))
d.add(elm.Line( "right", l=d.unit/8))

d.add(elm.Line( "right", l=d.unit/4, xy=[0, 1.5*d.unit], lftlabel='$IN_1$'))
Q1 = d.add(dsp.Circle())
d.add(elm.Line( "right", l=d.unit/4))

D4 = d.add(dsp.Square(label='$\int$'))
d.add(elm.Line( "right", l=d.unit/8))
d.add(elm.Line( "down", l=0.75*d.unit))
d.add(elm.Line( "right", l=d.unit*0.30))

S = d.add(dsp.Mixer(xy=[d.unit, 0.75*d.unit]))
d.add(elm.Line( "right", l=d.unit/4, rgtlabel='$OUT$'))


d.draw()


first we design the input stage. the input voltage for the X signal should be 10mV and 20mV for the control voltage.


In [ ]:
#load the kicad schema
directory_path = Path(os.path.abspath('')).resolve()
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))


In [ ]:
#ringmodulation
ringmod_schema = parser.build_circuit(ground=5)
ringmod_schema.include(spice_library['TL072c'])
ringmod_schema.include(spice_library['AD633'])

ringmod_schema.V('1', '+15V', ringmod_schema.gnd, 'DC 15')
ringmod_schema.V('2', '-15V', ringmod_schema.gnd, 'DC -15')
ringmod_schema.V('3', 'IN_Xa', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 1k)')
ringmod_schema.V('4', 'IN_Ya', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
ringmod_schema.V('5', 'IN_Za', ringmod_schema.gnd, 'DC 0V AC 0V')

#simulator = ringmod_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_ringmod = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#amplitude modulation
am_schema = parser.build_circuit(ground=5)
am_schema.include(spice_library['TL072c'])
am_schema.include(spice_library['AD633'])

am_schema.V('1', '+15V', am_schema.gnd, 'DC 15')
am_schema.V('2', '-15V', am_schema.gnd, 'DC -15')
am_schema.V('3', 'IN_Xa', am_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
am_schema.V('4', 'IN_Ya', am_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
am_schema.V('5', 'IN_Za', am_schema.gnd, 'DC 5V AC 5V SIN(0V 2.5V 1k)')

#simulator = am_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_am = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#vca
vca_schema = parser.build_circuit(ground=5)
vca_schema.include(spice_library['TL072c'])
vca_schema.include(spice_library['AD633'])

vca_schema.V('1', '+15V', vca_schema.gnd, 'DC 15')
vca_schema.V('2', '-15V', vca_schema.gnd, 'DC -15')
vca_schema.V('3', 'IN_Xa', vca_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
vca_schema.V('4', 'IN_Ya', vca_schema.gnd, 'DC 5V AC 5V PULSE(0V 10V 1m 1m 15m 1m)')
vca_schema.V('5', 'IN_Za', vca_schema.gnd, 'DC 0V AC 0V')

#simulator = vca_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_vca = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

# bom
{: class="subtitle is-lowercase mt-6"}

main circuit
{: class="is-size-6 has-text-weight-semibold mt-6"}

{% include bom.html content="summe-main-bom" %}
{% include callouts.html %}
{% include reports.html%}

# calibration
{: class="subtitle is-lowercase"}

there is no calibration needed. but the potentiomenter knobs have to be aliged to center position.

# usage
{: class="subtitle is-lowercase"}

the input jacks are wired to 5 volts when nothing is connected.

all channels are mixed to the out jack. when something is connected to the channel out, this channel is removed from the overall mix.

_mixer_
* connect the different channels from audio or cv sources to the in jacks.
* connect the out jack to something
* turn the pots clockwise to adjust the volume.

_attenuverter_
* connect all or a single channel. 
* when you turn the pot knob counter clockwise the signal is inverted.


# *references*
{: class="subtitle is-lowercase"}

- [VCA Techniques Investigated](https://sound-au.com/articles/vca-techniques.html) Rod Elliott (ESP).
- [Popular Electronics](https://tinaja.com/glib/pop_elec/mus_keying_vca_1+2_75.pdf) Keyiing and VCA citcuits for electronic music instruments.
- [VCA-1](https://www.birthofasynth.com/Thomas_Henry/Pages/VCA-1.html) Thomas Henry CA3080 VCA.
- [VCA-3](https://www.schmitzbits.de/vca3.png) René Schmitz differential pair VCA.
- [#223](https://www.youtube.com/watch?v=7nmmb0pqTU0&t=2s): Basics of the Gilbert Cell | Analog Multiplier | Mixer | Modulator
- [#224](https://www.youtube.com/watch?v=38OQub2Vi2Q): AM & DSB-SC Modulation with the Gilbert Cell
- [Analog Devices](https://www.analog.com/media/en/training-seminars/design-handbooks/ADI_Multiplier_Applications_Guide.pdf): Analog multiplier application guide 
- [AD633](https://www.analog.com/media/en/technical-documentation/data-sheets/AD633.pdf): Datasheet 
{: class="mb-6"}
